In [1]:
import pandas as pd
import numpy as np
import torch
import networkx as nx
import cobra
import matplotlib.pyplot as plt

import sys
sys.path.append("../src/")
import GEMtoGRAPH as gg

pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
model = cobra.io.load_json_model('redYeast_ST8943_fdp1.json')
S = cobra.util.array.create_stoichiometric_matrix(model, array_type='DataFrame')
S.shape

(300, 373)

# MFG

#### Load TFA fluxes

In [3]:
# load tfa fluxes and send them to graph construction functions
tfa = pd.read_csv('fluxes_for_graph.csv', index_col=0)
tfa = tfa.head(1)

zero_flux = [col for col in tfa.columns if (tfa[col] == 0).all()]

print('Zero flux reactions:',len(zero_flux))

tfa.drop(columns=zero_flux, inplace=True)
print("TFA fluxes:", tfa.shape[1])

# For _reverse reactions we should change the sign of the flux to negative
for col in tfa.columns:
    if '_reverse' in col: tfa[col] = -tfa[col]


tfa.rename(columns={col: col.split("_reverse_")[0] for col in tfa.columns}, inplace=True)

tfa_flux = tfa.iloc[0].values
tfa_flux = pd.DataFrame(columns=['fluxes'], data=tfa_flux)
tfa_flux.index = S.columns

Zero flux reactions: 373
TFA fluxes: 373


### Create MFG Graph

In [4]:
M, S_2m, G = gg.MFG(S, model, tfa_flux)

# Remove isolated nodes from G
print()
print('Removing isolated nodes...')
isolated_nodes = list(nx.isolates(G))
G.remove_nodes_from(isolated_nodes)

print("# nodes:", G.number_of_nodes(), "\n# edges:", G.number_of_edges())

# nodes: 746 
# edges: 6157

Removing isolated nodes...
# nodes: 373 
# edges: 6157


## Read ORACLE's data

In [5]:
sigma = pd.read_csv('saturations.csv', index_col=0)
gamma = pd.read_csv('gamma.csv', index_col=0)
vmax = pd.read_csv('Vmax_matrix.csv', index_col=0)

gamma = gamma.head(1)
sigma = sigma.head(1)
vmax = vmax.head(1)

In [6]:
# get the reactions that are the reversible 
rev_rxn = []
for node in list(G.nodes()):
    if node.split("?")[0] == 'rev': rev_rxn.append(node.split("?")[1])

# rename the reactions of gamma; if it's the reversible one add rev? to the column name
for col in gamma.columns:
    if col in rev_rxn: gamma.rename(columns={col:'rev?'+col}, inplace=True)

In [7]:
listA = list(G.nodes())
listB = gamma.columns

print('In Graph but not in gamma:', [item for item in listA if item not in listB])
print()
print('In gamma but not in Graph:', [item for item in listB if item not in listA])

In Graph but not in gamma: ['EX_lac__D_e', 'EX_mal__L_e', 'EX_akg_e', 'EX_2phetoh_e', 'EX_acald_e', 'EX_ac_e', 'EX_gam6p_e', 'EX_co2_e', 'EX_cit_e', 'EX_etoh_e', 'EX_fum_e', 'EX_gly_e', 'EX_gcald_e', 'EX_glx_e', 'EX_id3acald_e', 'EX_ala__L_e', 'EX_asn__L_e', 'EX_asp__L_e', 'EX_cys__L_e', 'EX_glu__L_e', 'EX_gln__L_e', 'EX_phe__L_e', 'EX_ser__L_e', 'EX_trp__L_e', 'EX_tyr__L_e', 'EX_oaa_e', 'EX_pacald_e', 'EX_pyr_e', 'EX_succ_e', 'EX_ind3eth_e', 'EX_h2o_e', 'EX_g6p_e', 'EX_g1p_e', 'EX_2pg_e', 'EX_pser__L_e', 'EX_ppi_e', 'EX_pep_e', 'EX_cbp_e', 'EX_6pgc_e', 'EX_3pg_e', 'EX_cmp_e', 'GROWTH', 'EX_ccm_e', 'EX_pca_e', 'rev?EX_nh4_e', 'rev?EX_glc__D_e', 'rev?EX_h_e', 'rev?EX_fe2_e', 'rev?EX_o2_e', 'rev?EX_pi_e', 'rev?EX_k_e', 'rev?EX_na1_e', 'rev?EX_so4_e', 'rev?EX_cl_e', 'rev?EX_cu2_e', 'rev?EX_mn2_e', 'rev?EX_zn2_e', 'rev?EX_mg2_e', 'rev?EX_ca2_e']

In gamma but not in Graph: []


#### Add `gamma` values as Graph node features

In [8]:
for node in gamma.columns:
    try:
        G.nodes[node]['gamma'] =  gamma[node].values[0]
    except KeyError:
        pass

no_gamma_nodes = [node for node, data in G.nodes(data=True) if not data]

for node in no_gamma_nodes: G.nodes[node]['gamma'] = np.nan

### We have the `Networkx` Graph G

In [9]:
G.number_of_nodes(), G.number_of_edges()

(373, 6157)

In [10]:
## Create features based on graph statistics such as degree_centrality etc

df_g = pd.DataFrame(index=G.nodes())
df_g['clustering'] = pd.Series(nx.clustering(G))
df_g['in_degree'] = pd.Series(nx.in_degree_centrality(G))
df_g['out_degree'] = pd.Series(nx.out_degree_centrality(G))
df_g['degree_centrality'] = pd.Series(nx.degree_centrality(G))
df_g['closeness'] = pd.Series(nx.closeness_centrality(G))
df_g['betweeness'] = pd.Series(nx.betweenness_centrality(G))
df_g['pr'] = pd.Series(nx.pagerank(G))

node_data = pd.DataFrame.from_dict(dict(G.nodes), orient='index')
node_data.sort_index(inplace=True)

df_g.sort_index(inplace=True)

df_g['gamma'] = node_data['gamma'].copy()
df_g.head()

,clustering,in_degree,out_degree,degree_centrality,closeness,betweeness,pr,gamma
2OBUTtm,0.00000,0.00269,0.00269,0.00538,0.31560,0.00000,0.00060,0.99900
2OXOADPTm,0.04000,0.03495,0.01344,0.04839,0.33442,0.00165,0.00079,0.99900
2PHETOHtm,0.00000,0.00269,0.00269,0.00538,0.25402,0.00019,0.00102,0.99322
6PHOPHO,0.14286,0.00538,0.01075,0.01613,0.22767,0.00002,0.00080,0.99900
AATA,0.10780,0.05645,0.02957,0.08602,0.31280,0.00129,0.00106,0.30862


In [11]:
df_g.shape

(373, 8)

### Drop lines (reactions) with `gamma` > 1

In [16]:
# drop these lines because of gamma > 1
index_to_drop = df_g[df_g['gamma'] > 1].index

df_g.drop(index=index_to_drop, inplace=True)

### Add more node features using data from GEM

### Create a _node features_ matrix

In [17]:
df = pd.DataFrame(index=list(G.nodes()), columns=['compartements', 'metabolites', 'num_of_mets'])

for rxn in df.index:
    if 'rev?' in rxn: rxn = rxn.split("?")[1]
    else: rxn = rxn

    metabolites = []
    for m in model.reactions.get_by_id(rxn).metabolites: metabolites.append(m.id)
  
    metabolites = "|".join(metabolites)
    compartements = "|".join(list(model.reactions.get_by_id(rxn).compartments))

    # compartements = list(model.reactions.get_by_id(rxn).compartments)
    
    num_of_mets = len(model.reactions.get_by_id(rxn).metabolites)
    new_row = [compartements, metabolites, num_of_mets]

    df.loc[rxn] = new_row

df.head(3)

,compartements,metabolites,num_of_mets
D_LACDcm,m|c,ficytc_m|focytc_m|lac__D_c|pyr_c,4
D_LACDm,m,ficytc_m|focytc_m|lac__D_m|pyr_m,4
L_LACD2cm,m|c,ficytc_m|focytc_m|lac__L_c|pyr_c,4


In [18]:
# LABEL ENCODING to every compartement and metabolite
from sklearn.preprocessing import LabelEncoder

def enc_for_every():
    COBRA_METABOLITES = pd.DataFrame([m.id for m in model.metabolites])
    COBRA_METABOLITES.rename(columns = {0:'id'}, inplace = True)
    COBRA_METABOLITES['enc'] = LabelEncoder().fit_transform(COBRA_METABOLITES['id'])

    COBRA_COMPARTEMETNS = pd.DataFrame(list(model.compartments.keys()))
    COBRA_COMPARTEMETNS.rename(columns= {0:'id'}, inplace=True)
    COBRA_COMPARTEMETNS['enc'] = LabelEncoder().fit_transform(COBRA_COMPARTEMETNS['id'])

    for row in range(len(df)):

        c = df.iloc[row]['compartements']
        m = df.iloc[row]['metabolites']

        try:
            df.loc[df.index[row], 'compartements'] = ([dict(COBRA_COMPARTEMETNS.values).get(item, item) for item in c])
            df.loc[df.index[row], 'metabolites'] = ([dict(COBRA_METABOLITES.values).get(item, item) for item in m])
        except TypeError:
            pass

# Encoding for c|m -> 0, ... 
df['compartements'] = LabelEncoder().fit_transform(df['compartements'])

In [19]:
# rename 'gamma' feature name to 'y'
for node, data in G.nodes(data=True):

    data['y'] = data.pop('gamma')

In [20]:
for node in list(G.nodes()):
    if 'rev?' in node: rxn = node.split("?")[1]
    else: rxn = node

    # nx.set_node_attributes(G, {node: {'compartements':df.loc[rxn]['compartements']}})
    nx.set_node_attributes(G, {node: {'num_of_mets':df.loc[rxn]['num_of_mets']}})

## Networkx to Torch Geometric

In [59]:
import torch
import torch.nn as nn
from torch_geometric.utils.convert import from_networkx

device = 'cuda' if torch.cuda.is_available() else 'cpu'

data = from_networkx(G, group_edge_attrs=all)

print(data)
print()
print(data.num_nodes ,data.num_edges)

Data(edge_index=[2, 6157], y=[373], num_of_mets=[373], edge_attr=[6157, 1], num_nodes=373)

373 6157


In [60]:
data.x = data.num_of_mets
del data.num_of_mets

data.x = torch.tensor(data.x, dtype=torch.float)
data

Data(edge_index=[2, 6157], y=[373], edge_attr=[6157, 1], num_nodes=373, x=[373])

In [63]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing

class GNN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='mean')
        self.lin = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.lin(x)
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        return x_j

    def update(self, aggr_out, x):
        return aggr_out

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.gnn = GNN(1, 128)
        self.fc = nn.Linear(128, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gnn(x, edge_index)
        x = self.fc(x)
        return x.view(-1)

# Define the model, loss function and optimizer
model = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    running_loss = 0.0
    for data in train_dataloader:
        optimizer.zero_grad()
        outputs = model(data)
        labels = data.y
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("Epoch {} loss: {:.4f}".format(epoch + 1, running_loss / len(train_dataloader)))

# Evaluate the model on the test set
with torch.no_grad():
    running_loss = 0.0
    for data in test_dataloader:
        outputs = model(data)
        labels = data.y
        loss = criterion(outputs, labels)
        running_loss += loss.item()
    print("Test loss: {:.4f}".format(running_loss / len(test_dataloader)))

NameError: name 'train_dataset' is not defined

In [39]:
import torch
from torch.nn import Linear
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
embedding_size = 64

class GCN(torch.nn.Module):
    def __init__(self):
        # Init parent
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # GCN layers
        self.initial_conv = GCNConv(data.num_features, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)

        # Output layer
        self.out = Linear(embedding_size*2, 1)

    def forward(self, x, edge_index, batch_index):
        # First Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = F.tanh(hidden)

        # Other Conv layers
        hidden = self.conv1(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv2(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv3(hidden, edge_index)
        hidden = F.tanh(hidden)
          
        # Global Pooling (stack different aggregations)
        hidden = torch.cat([gmp(hidden, batch_index), 
                            gap(hidden, batch_index)], dim=1)

        # Apply a final (linear) classifier.
        out = self.out(hidden)

        return out, hidden

model = GCN()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GCN(
  (initial_conv): GCNConv(1, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Number of parameters:  12737


In [57]:
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

# Root mean squared error
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007)  

# Use GPU for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Wrap data in a data loader
data_size = len(data.x)
NUM_GRAPHS_PER_BATCH = 64
loader = DataLoader(data.x[:int(data_size * 0.8)], 
                    batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_loader = DataLoader(data.x[int(data_size * 0.8):], 
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)


In [58]:
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

# Root mean squared error
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007)  

# Use GPU for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Wrap data in a data loader
data_size = len(data)
NUM_GRAPHS_PER_BATCH = 64
loader = DataLoader(data[:int(data_size * 0.8)], 
                    batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_loader = DataLoader(data[int(data_size * 0.8):], 
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)

def train(data):
    # Enumerate over the data
    for batch in loader:
      # Use GPU
      batch.to(device)  
      # Reset gradients
      optimizer.zero_grad() 
      # Passing the node features and the connection info
      pred, embedding = model(batch.x.float(), batch.edge_index, batch.batch) 
      # Calculating the loss and gradients
      loss = loss_fn(pred, batch.y)     
      loss.backward()  
      # Update using the gradients
      optimizer.step()   
    return loss, embedding

print("Starting training...")
losses = []
for epoch in range(2000):
    loss, h = train(data)
    losses.append(loss)
    if epoch % 100 == 0:
      print(f"Epoch {epoch} | Train Loss {loss}")

TypeError: unhashable type: 'slice'

In [56]:
# Initialize Optimizer
learning_rate = 0.01
decay = 5e-4
# Define loss function (CrossEntropyLoss for Classification Problems with 
# probability distributions)
criterion = torch.nn.MSELoss()

def train():
      model.train()
      optimizer.zero_grad() 
      # Use all data as input, because all nodes have node features
      out = model(data.x, data.edge_index)  
      # Only use nodes with labels available for loss calculation --> mask
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  
      loss.backward() 
      optimizer.step()
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      # Use the class with highest probability.
      pred = out.argmax(dim=1)  
      # Check against ground-truth labels.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  
      # Derive ratio of correct predictions.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  
      return test_acc

losses = []
for epoch in range(0, 1001):
    loss = train()
    losses.append(loss)
    if epoch % 100 == 0:
      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

TypeError: forward() missing 1 required positional argument: 'batch_index'

In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

embedding_size = 64

class GCN(torch.nn.Module):
    def __init__(self):
        # Init parent
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # GCN layers
        self.initial_conv = GCNConv(data.num_features, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)

        # Output layer
        self.out = Linear(embedding_size*2, 1)

    def forward(self, x, edge_index, batch_index):
        # First Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = F.tanh(hidden)

        # Other Conv layers
        hidden = self.conv1(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv2(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv3(hidden, edge_index)
        hidden = F.tanh(hidden)
          
        # Global Pooling (stack different aggregations)
        hidden = torch.cat([gmp(hidden, batch_index), 
                            gap(hidden, batch_index)], dim=1)

        # Apply a final (linear) classifier.
        out = self.out(hidden)

        return out, hidden

model = GCN()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

In [ ]:
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

# Root mean squared error
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007)  

# Use GPU for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Split the data into training and test sets
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])

# Create DataLoader objects for the training and test sets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:

def train(data):
    # Enumerate over the data
    for batch in loader:
      # Use GPU
      batch.to(device)  
      # Reset gradients
      optimizer.zero_grad() 
      # Passing the node features and the connection info
      pred, embedding = model(batch.x.float(), batch.edge_index, batch.batch) 
      # Calculating the loss and gradients
      loss = loss_fn(pred, batch.y)     
      loss.backward()  
      # Update using the gradients
      optimizer.step()   
    return loss, embedding

print("Starting training...")
losses = []
for epoch in range(2000):
    loss, h = train(data)
    losses.append(loss)
    if epoch % 100 == 0:
      print(f"Epoch {epoch} | Train Loss {loss}")

In [ ]:
import torch.nn.functional as F

" **************** CONSTRUCT THE MODEL  ********************"
SGC_model = SGConv(in_channels= data.num_features, # Number of features
                   out_channels= 1, # Dimension of embedding
                   K = 1)

" **************** GET EMBEDDING  ********************"
print(" Shape of the original data: ", data.x.shape)
print(" Shape of the embedding data: ", SGC_model(data.x,data.edge_index).shape)


" **************** CONSTRUCT THE MODEL FOR REGRESSION  ********************"
class SGCNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SGConv(in_channels= data.num_features, # Number of features
                   out_channels= 1, # Dimension of embedding
                   K = 1)
 
    def forward(self):
        x = self.conv1(data.x,  data.edge_index) #Applying convolution to data
        return x
    

SGC_model, data = SGCNet().to(device), data.to(device)

optimizer = torch.optim.Adam(SGC_model.parameters(), lr=0.2, weight_decay=0.005)

# What are the learning parameters:
for i, parameter in SGC_model.named_parameters():
    print(" Parameter {}".format(i))
    print("Shape: ",parameter.shape)

In [ ]:
" **************** TRAIN FUNCTION ********************"
def train():
    SGC_model.train() # Set the model.training to be True
    optimizer.zero_grad() # Reset the gradient
    predicted_y = SGC_model() # predicted y in log softmax prob
    true_y = data.y # True labels
    losses = F.mse_loss(predicted_y[data.train_mask], true_y[data.train_mask])
    losses.backward()
    optimizer.step() # Update the parameters such that is minimized the losses
    
    
" **************** TEST FUNCTION ********************"
def test():
    SGC_model.eval() # Set the model.training to be False
    logits = SGC_model() # Log prob of all data
    accs = []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1] #Transforming log prob to actual labels
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs



" **************** PUTTING IT ALL TOGETHER ********************"
best_val_acc = test_acc = 0
for epoch in range(1, 101):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, train_acc, best_val_acc, test_acc))